Critique du film Inception sur Allociné
http://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/

In [2]:
import requests
import lxml.html as lh
import pandas as pd

In [3]:
# URL
url = 'http://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/'
# URI-SUFFIX
uri_pages = '?page='
# Number of pages to read
nbPages = 400
# XPath content to collect
tags = ['//span[@class="stareval-note"]', \
        '//div[@class="content-txt review-card-content"]' ]
cols = ['Note', \
        'Description' ]

In [4]:
page = requests.get(url)
doc = lh.fromstring(page.content)

In [5]:
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)

    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))

    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)

    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
    
    return liste

In [6]:
def getPages(_nbPages, _url):
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        liste = getPage(_url + uri_pages + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale

liste_totale = getPages(nbPages, url)
liste_totale.head()

,Note,Description,key
0,"5,0",Après le chef d'oeuvre super-héroïque The Dark...,0
1,"5,0",Souvent l’on peut manquer d’inspiration ou de ...,1
2,"5,0",CHEF D’ŒUVRE ! Le film est absolument parfait ...,2
3,"3,5",Le meilleur blockbuster de 2010 a pour thème l...,3
4,"5,0","Un film aussi novateur que complexe, dont la m...",4


Retire les caractères sans interet ainsi que les retours chariots

In [15]:
import csv
import re
REMPLACE_SANS_ESPACE = re.compile("[;:!\'?,\"()\[\]]")
REMPLACE_AVEC_ESPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|[.]")

def preprocess(txt):
    txt = [line.replace('\n', ' ')  for line in txt] # Retire les \n (retours chariots)
    txt = [REMPLACE_SANS_ESPACE.sub("", line.lower()) for line in txt]
    txt = [REMPLACE_AVEC_ESPACE.sub(" ", line) for line in txt]
    return txt
liste_totale['Description'] = pd.DataFrame(preprocess(liste_totale['Description']))

In [16]:
liste_totale.to_csv('../../datasources/films/allocine_inception_avis.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [17]:
liste_totale

,Note,Description,key
0,"5,0",après le chef doeuvre super héroïque the dark ...,0
1,"5,0",souvent l’on peut manquer d’inspiration ou de ...,1
2,"5,0",chef d’œuvre le film est absolument parfait ...,2
3,"3,5",le meilleur blockbuster de 2010 a pour thème l...,3
4,"5,0",un film aussi novateur que complexe dont la mi...,4
...,...,...,...
5995,"5,0",je nai quune chose à dire allez le voir non ...,10
5996,"5,0",tout simplement fantastique très bon scénario...,11
5997,"4,5",cest beau cest bien fait ça se regarde vraimen...,12
5998,"5,0",pour moi le meilleur film de sf que jai vue l...,13


In [18]:
pd.read_csv('../../datasources/films/allocine_inception_avis.csv')

,Note,Description,key
0,"5,0",après le chef doeuvre super héroïque the dark ...,0
1,"5,0",souvent l’on peut manquer d’inspiration ou de ...,1
2,"5,0",chef d’œuvre le film est absolument parfait ...,2
3,"3,5",le meilleur blockbuster de 2010 a pour thème l...,3
4,"5,0",un film aussi novateur que complexe dont la mi...,4
...,...,...,...
5995,"5,0",je nai quune chose à dire allez le voir non ...,10
5996,"5,0",tout simplement fantastique très bon scénario...,11
5997,"4,5",cest beau cest bien fait ça se regarde vraimen...,12
5998,"5,0",pour moi le meilleur film de sf que jai vue l...,13
